# Código para Planeación hacia Atrás

In [155]:
import copy
from typing import List, Callable, Dict

class Proposition:
    def __init__(self, name:str, arg:str = '', negation:bool = False):
        self.name = name
        self.argument = arg
        self.negation = negation
        
    def __hash__(self):
        return hash((self.name, self.argument, self.negation))
    
    def __eq__(self, other):
        return isinstance(other, Proposition) and self.name == other.name and self.argument == other.argument and self.negation == other.negation
    
    def __invert__(self):
        if self.negation:
            return Proposition(self.name, self.argument, False)
        else:
            return Proposition(self.name, self.argument, True)
    
    def __str__(self):
        if self.negation:
            return f'¬{self.name}({self.argument})'
        else:
            return f'{self.name}({self.argument})'

# Se añadió el método de regresión para hacer la búsqueda sobre el espacio de objetivos
class Action:
    def __init__(self, name:str, preconditions:List[Proposition], delete:List[Proposition], effect:List[Proposition], variants:List[Proposition] = []):
        self.name = name
        self.preconditions = preconditions
        self.delete = delete
        self.effect = effect
        self.variants = variants
        
    def apply(self, state):
        state = copy.deepcopy(state)
        
        if len(self.variants) > 0:
            for condition, variant_effect, variant_delete in self.variants:
                if all(precondition in state for precondition in self.preconditions):
                    if condition in state: 
                        for element in variant_delete:
                            if element in state:
                                state.remove(element)
                        for element in variant_effect:
                            state.append(element)
                    else:
                        for element in self.delete:
                            if element in state:
                                state.remove(element)
                        for element in self.effect:
                            state.append(element)
        else:
            if all(precondition in state for precondition in self.preconditions):
                for element in self.delete:
                    if element in state:
                        state.remove(element)
                for element in self.effect:
                    state.append(element)
        return list(set(state))
    
    def regression(self, state:list):
        # Se verifica si el estado es regresable por la acción,
        # verificando si no existe una consistente con lo que se 
        # añade o se quita debido a la acción
        for remove_proposition in self.delete:
            if remove_proposition in state:
                return []
        for added_proposition in self.effect:
            if added_proposition not in state:
                return []
        
        # Se forma la regresión. De manera general, se agregan 
        # las proposiciones que la acción no añade y las precondiciones 
        # de la acción para poder activarse 
        regression_state = []
        # Cuando tiene variantes 
        if len(self.variants) > 0:
            for condition, variant_effect, variant_delete in self.variants:
                for proposition in variant_delete:
                    if proposition in state:
                        break
                for proposition in variant_effect:
                    if proposition not in state:
                        break
                for proposition in state:
                    if proposition not in variant_effect:
                        regression_state.append(proposition)
                regression_state.append(condition)
        else:
            for proposition in state:
                if proposition not in self.effect:
                    regression_state.append(proposition)
            for precondition in self.preconditions:
                regression_state.append(precondition)
            for proposition in self.delete:
                regression_state.append(proposition)

        return list(set(regression_state))

    def __str__(self):
        return f'{self.name}'

class Node:
    def __init__(self, state:List[Proposition], cost:int, heuristic:int, plan:str):
        self.state = state
        self.cost = cost
        self.heuristic = heuristic
        self.function = cost + heuristic
        self.plan = plan

def build_plan_graph(state:List[Proposition], goal:List[Proposition], actions:List[Action]) -> List[List[Proposition]]:   
    graph_levels = [state]
    
    i = 0
    while(True):
        new_level = [*graph_levels[i]]
        
        for action in actions:
            new_literals = action.apply(graph_levels[i])
            for lit in new_literals:
                new_level.append(lit)
                
        graph_levels.append(list(set(new_level)))
        
        if all(elem in new_level for elem in goal):
            break
        
        if len(list(set(new_level))) == len(graph_levels[i]):
            return []
            break
        
        i += 1
            
    return graph_levels

def graph_heuristic_backward(state:List[Proposition],planning_graph:List[List[Proposition]]) -> int:
    # Lo que se quiere es una heurística que indique en que nivel del grafo de 
    # planificación se encuentran todas las proposiciones de state
    level = 0
    try:
        while not all(proposition in planning_graph[level] for proposition in state):
            level += 1
        return level
    except:
        return float('inf')

def backward_A_star(initial_node:Node, goal:List[Proposition], actions:List[Action]) -> str:
    # Se calcula la gráfica de planeación del estado inicial que se usará como heurística
    planning_graph = build_plan_graph(initial_node.state,goal,actions)

    frontier = [initial_node]
    while len(frontier) > 0:
        node = frontier.pop(0)
        
        if SameList(node.state,goal):
            # Devuelve la lista de acciones en el orden 
            # inverso a como se agregaron y esta es el plan
            return node.plan[::-1]
            
        else:
            for action in actions:
                # Los nuevos estados son las regresiones del estado actual
                new_state = action.regression(node.state)
                   
                if all(elem in node.state for elem in new_state) and all(elem in new_state for elem in node.state) :
                    pass
                else:
                    h = graph_heuristic_backward(new_state,planning_graph)
                    c = node.cost + 1
                    new_node = Node(new_state, c, h, copy.deepcopy(node.plan))
                    new_node.plan.append(action)
                    
                    frontier.append(new_node)
            
            frontier = sorted(frontier, key = lambda x: x.function)
            
    return 'THERE IS NOT A FEASIBLE PLAN'

def SameList(state:List[Proposition],goal:List[Proposition]):
    # Esta función permite decidir cuando dos listas son iguales
    # tanto en tamaño como los elementos que contienen 
    if len(state) == len(goal):
        return all(proposition in state for proposition in goal)
    else:
        return False

# Problemas

Para que funcione adecuadamente el algoritmo de planeación hacia atrás es necesario añadir otras proposiciones al objetivo, que son consecuencia del propio estado objetivo. Ejemplificando esto:
* En el problema de los Bloques: El bloque C está sobre la mesa, no tenemos nada en la mano y no hay ningún bloque encima de A, todo esto es consistente con el objetivo, pero no es necesario declararlo
* En el problema del Robot: No fue necesario añadir ninguna proposición extra al objetivo
* En el problema de la Planeación de Cena: Se añadió que se tienen las manos sucias, esto es consecuencia de sacar la basura y que no es parte del objetivo pero sí es consistente con el objetivo

## Problema de los Bloques

In [ ]:
pickUpA = Action(
    'Pick Up A', 
    [Proposition('OnTable', 'A'), Proposition('Clear', 'A'), Proposition('HandEmpty')],            
    [Proposition('OnTable', 'A'), Proposition('Clear', 'A'), Proposition('HandEmpty')], 
    [Proposition('Holding', 'A')],
)

pickUpB = Action(
    'Pick Up B', 
    [Proposition('OnTable', 'B'), Proposition('Clear', 'B'), Proposition('HandEmpty')],            
    [Proposition('OnTable', 'B'), Proposition('Clear', 'B'), Proposition('HandEmpty')], 
    [Proposition('Holding', 'B')],
)

pickUpC = Action(
    'Pick Up C', 
    [Proposition('OnTable', 'C'), Proposition('Clear', 'C'), Proposition('HandEmpty')],            
    [Proposition('OnTable', 'C'), Proposition('Clear', 'C'), Proposition('HandEmpty')], 
    [Proposition('Holding', 'C')],
)

putDownA = Action(
    'Put down A', 
    [Proposition('Holding', 'A')],            
    [Proposition('Holding', 'A')], 
    [Proposition('OnTable', 'A'), Proposition('Clear', 'A'), Proposition('HandEmpty')],
)

putDownB = Action(
    'Put down B', 
    [Proposition('Holding', 'B')],            
    [Proposition('Holding', 'B')], 
    [Proposition('OnTable', 'B'), Proposition('Clear', 'B'), Proposition('HandEmpty')],
)

putDownC = Action(
    'Put down C', 
    [Proposition('Holding', 'C')],            
    [Proposition('Holding', 'C')], 
    [Proposition('OnTable', 'C'), Proposition('Clear', 'C'), Proposition('HandEmpty')],
)

stackAB = Action(
    'Stack A on B', 
    [Proposition('Holding', 'A'), Proposition('Clear', 'B')],            
    [Proposition('Holding', 'A'), Proposition('Clear', 'B')], 
    [Proposition('On', 'A,B'), Proposition('Clear', 'A'), Proposition('HandEmpty')],
)

stackBA = Action(
    'Stack B on A', 
    [Proposition('Holding', 'B'), Proposition('Clear', 'A')],            
    [Proposition('Holding', 'B'), Proposition('Clear', 'A')], 
    [Proposition('On', 'B,A'), Proposition('Clear', 'B'), Proposition('HandEmpty')],
)

stackAC = Action(
    'Stack A on C', 
    [Proposition('Holding', 'A'), Proposition('Clear', 'C')],            
    [Proposition('Holding', 'A'), Proposition('Clear', 'C')], 
    [Proposition('On', 'A,C'), Proposition('Clear', 'A'), Proposition('HandEmpty')],
)

stackCA = Action(
    'Stack C on A', 
    [Proposition('Holding', 'C'), Proposition('Clear', 'A')],            
    [Proposition('Holding', 'C'), Proposition('Clear', 'A')], 
    [Proposition('On', 'C,A'), Proposition('Clear', 'C'), Proposition('HandEmpty')],
)

stackBC = Action(
    'Stack B on C', 
    [Proposition('Holding', 'B'), Proposition('Clear', 'C')],            
    [Proposition('Holding', 'B'), Proposition('Clear', 'C')], 
    [Proposition('On', 'B,C'), Proposition('Clear', 'B'), Proposition('HandEmpty')],
)

stackCB = Action(
    'Stack C on B', 
    [Proposition('Holding', 'C'), Proposition('Clear', 'B')],            
    [Proposition('Holding', 'C'), Proposition('Clear', 'B')], 
    [Proposition('On', 'C,B'), Proposition('Clear', 'C'), Proposition('HandEmpty')],
)

unstackAB = Action(
    'Unstack A from B', 
    [Proposition('On', 'A,B'), Proposition('Clear', 'A'), Proposition('HandEmpty')],            
    [Proposition('On', 'A,B'), Proposition('Clear', 'A'), Proposition('HandEmpty')], 
    [Proposition('Holding', 'A'), Proposition('Clear', 'B')],
)

unstackBA = Action(
    'Unstack B from A', 
    [Proposition('On', 'B,A'), Proposition('Clear', 'B'), Proposition('HandEmpty')],            
    [Proposition('On', 'B,A'), Proposition('Clear', 'B'), Proposition('HandEmpty')], 
    [Proposition('Holding', 'B'), Proposition('Clear', 'A')],
)

unstackAC = Action(
    'Unstack A from C', 
    [Proposition('On', 'A,C'), Proposition('Clear', 'A'), Proposition('HandEmpty')],            
    [Proposition('On', 'A,C'), Proposition('Clear', 'A'), Proposition('HandEmpty')], 
    [Proposition('Holding', 'A'), Proposition('Clear', 'C')],
)

unstackCA = Action(
    'Unstack C from A', 
    [Proposition('On', 'C,A'), Proposition('Clear', 'C'), Proposition('HandEmpty')],            
    [Proposition('On', 'C,A'), Proposition('Clear', 'C'), Proposition('HandEmpty')], 
    [Proposition('Holding', 'C'), Proposition('Clear', 'A')],
)

unstackBC = Action(
    'Unstack B from C', 
    [Proposition('On', 'B,C'), Proposition('Clear', 'B'), Proposition('HandEmpty')],            
    [Proposition('On', 'B,C'), Proposition('Clear', 'B'), Proposition('HandEmpty')], 
    [Proposition('Holding', 'B'), Proposition('Clear', 'C')],
)

unstackCB = Action(
    'Unstack C from B', 
    [Proposition('On', 'C,B'), Proposition('Clear', 'C'), Proposition('HandEmpty')],            
    [Proposition('On', 'C,B'), Proposition('Clear', 'C'), Proposition('HandEmpty')], 
    [Proposition('Holding', 'C'), Proposition('Clear', 'B')],
)

actions = [
    pickUpA,
    pickUpB,
    pickUpC,
    putDownA,
    putDownB,
    putDownC,
    stackAB,
    stackBA,
    stackAC,
    stackCA,
    stackBC,
    stackCB,
    unstackAB,
    unstackBA,
    unstackAC,
    unstackCA,
    unstackBC,
    unstackCB
]

initial_state = [Proposition('On', 'C,A'), Proposition('OnTable', 'A'), Proposition('OnTable', 'B'), Proposition('Clear', 'C'), Proposition('Clear', 'B'), Proposition('HandEmpty')]
# Se ocupa añadir la preposición Proposition('Clear', 'A') , Proposition('HandEmpty') , Proposition('OnTable', 'C') para que funcione correctamente, por cómo se definió la regresión
goal_state = [Proposition('On', 'A,B'), Proposition('On','B,C'), Proposition('Clear', 'A'), Proposition('HandEmpty'), Proposition('OnTable', 'C')]

goal_node = Node(goal_state, 0, float('inf'), [])

[print(action) for action in backward_A_star(goal_node,initial_state,actions) if type(action) != str];

Unstack C from A
Put down C
Pick Up B
Stack B on C
Pick Up A
Stack A on B


## Problema del Robot

In [ ]:
grasp_key = Action(
    'Grasp Key', 
    [Proposition('In', 'Robot, R2'), Proposition('In', 'Key, R2')],            
    [], 
    [Proposition('Holding', 'Key')],
)

lock_door = Action(
    'Lock Door', 
    [Proposition('Holding', 'Key'), ~Proposition('Locked', 'Door')],
    [~Proposition('Locked', 'Door')],
    [Proposition('Locked', 'Door')],
)

key_into_box = Action(
    'Put Key Into Box',
    [Proposition('In', 'Robot, R1'), Proposition('In', 'Key, R1'), Proposition('Holding', 'Key')],
    [Proposition('Holding', 'Key'),  Proposition('In', 'Key, R1')],
    [Proposition('In', 'Key, Box')],
)

r1_to_r2 = Action(
    'Move from R1 to R2',
    [~Proposition('Locked', 'Door'), Proposition('In', 'Robot, R1')],
    [Proposition('In', 'Robot, R1')],
    [Proposition('In', 'Robot, R2')],
    [(Proposition('Holding', 'Key'), [Proposition('In', 'Robot, R2'), Proposition('In', 'Key, R2')], [Proposition('In', 'Robot, R1'), Proposition('In', 'Key, R1')])]
)

r2_to_r1 = Action(
    'Move from R2 to R1',
    [~Proposition('Locked', 'Door'), Proposition('In', 'Robot, R2')],
    [Proposition('In', 'Robot, R2')],
    [Proposition('In', 'Robot, R1')],
    [(Proposition('Holding', 'Key'), [Proposition('In', 'Robot, R1'), Proposition('In', 'Key, R1')], [Proposition('In', 'Robot, R2'), Proposition('In', 'Key, R2')])]
)

actions = [grasp_key, lock_door, key_into_box, r1_to_r2, r2_to_r1]

initial_state = [Proposition('In', 'Robot, R2'), Proposition('In', 'Key, R2'), ~Proposition('Locked', 'Door')]

goal_state = [Proposition('Locked', 'Door'), Proposition('In', 'Key, Box')]
goal_node = Node(goal_state, 0, float('inf'), [])

[print(action) for action in backward_A_star(goal_node,initial_state,actions)];

Grasp Key
Move from R2 to R1
Lock Door
Put Key Into Box


## Problema de la Planeación de Cena

In [ ]:
cook = Action(
    'Cook', 
    [Proposition('CleanHands')],            
    [], 
    [Proposition('Dinner')],
)

wrap = Action(
    'Wrap present', 
    [Proposition('Quiet')],            
    [], 
    [Proposition('Present')],
)

carry = Action(
    'Carry garbage', 
    [],            
    [Proposition('Garbage'), Proposition('CleanHands')], 
    [~Proposition('Garbage'), ~Proposition('CleanHands')],
)

dolly = Action(
    'Dolly garbage', 
    [],            
    [Proposition('Garbage'), Proposition('Quiet')], 
    [~Proposition('Garbage'), ~Proposition('Quiet')],
)

actions = [cook, wrap, carry, dolly]

initial_state = [Proposition('CleanHands'), Proposition('Quiet'), Proposition('Garbage')]
# Se ocupa añadir la preposición ~Proposition('CleanHands') para que funcione correctamente, por cómo se definió la regresión
goal_state = [Proposition('Dinner'), Proposition('Present'), ~Proposition('Garbage'),~Proposition('CleanHands')]
goal_node = Node(goal_state, 0, float('inf'), [])

[print(action) for action in backward_A_star(goal_node,initial_state,actions) if type(action) != str];

Cook
Carry garbage
Wrap present
